### Load up selenium and the Chrome webdriver

In [20]:
import selenium
from selenium import webdriver
from selenium.common import NoSuchElementException, ElementNotInteractableException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

In [21]:
driver = webdriver.Chrome()

In [22]:
driver.get('https://generalssb-prod.ec.njit.edu/BannerExtensibility/customPage/page/stuRegCrseSched')

### In the left navigation, find the 18th entry ("DS") and click on it

#### This will cause the main content area to load all of the DS courses

In [25]:
driver.implicitly_wait(5)
left_link = driver.find_element(By.ID, 'pbid-subjListTableSubjectLink-18')

In [26]:
ds_link = left_link.find_element(by=By.TAG_NAME, value='a')
ds_link.click()

In [27]:
my_span = driver.find_element(by=By.ID, value='pbid-courseListSectionDetailSections-0')

In [28]:
print(my_span)

<selenium.webdriver.remote.webelement.WebElement (session="6b67a1f81215e6b9464349ba695f82d7", element="f.B6096522AE51204AC200581E23D061CB.d.86852613547E7A98FFD71BDCC0313DF5.e.49")>


In [29]:
course_headers = my_span.find_elements(by=By.TAG_NAME, value="H4")

In [30]:
course_tables = my_span.find_elements(by=By.TAG_NAME, value="table")

In [31]:
print("I have {} headers and {} tables".format(len(course_headers), len(course_tables)))

I have 18 headers and 18 tables


In [32]:
def create_column_headings(table_headers):
    loc_columns = ['Course_name']
    for i in range(len(table_headers)):
        loc_columns.append(table_headers[i].text)
    return loc_columns

In [33]:
def create_course_listings(c_headers, c_tables):
    c_list = []
    for i in range(len(c_headers)):
        course_name = c_headers[i].text
#       collect the rows for each section of the course
        t_rows = c_tables[i].find_elements(by=By.CLASS_NAME, value='success')
#       for each row, get all of the cells
        for k in range(len(t_rows)):
            course_row = []
            course_row.append(course_name)
            t_cells = t_rows[k].find_elements(by=By.TAG_NAME, value='td')
            for l in range(len(t_cells)):
                course_row.append(t_cells[l].text)
            c_list.append(course_row)
    return c_list


In [34]:
# create column headings by using the 'th' cells of the first table
t_headers = course_tables[0].find_elements(by=By.TAG_NAME, value="th")
columns = create_column_headings(t_headers)

In [35]:
# use the list of course_headers and the list of course_tables to build
#   individual rows - one for each course/section
course_list = create_course_listings(course_headers, course_tables)
df = pd.DataFrame(course_list, columns=columns)
driver.quit()

In [36]:
df

,Course_name,Section,CRN,Days,Times,Location,Status,Max,Now,Instructor,Delivery Mode,Credits,Info,Comments
0,DS 340 - FUND. AND PRIN. OF DS,004,16164,MW,10:00 AM - 11:20 AM,CKB 120,Open,38,11,"Bhavsar, Jaini",Face-to-Face,3,Book,
1,DS 488 - INDEPENDENT STUDY IN DS,531,16403,,TBA,,Open,19,0,"Bader, David",Face-to-Face,3,Book,See department for permission to enroll.
2,DS 488 - INDEPENDENT STUDY IN DS,610,15926,,TBA,,Open,19,0,"Dasgupta, Aritra",Face-to-Face,3,Book,See department for permission to enroll.
3,DS 636 - DATA ANALYTICS WITH R PROGRAM,002,12240,M,1:00 PM - 3:50 PM,KUPF 210,Open,43,7,"Bhavsar, Jaini",Face-to-Face,3,Book,Students must bring their own device for this ...
4,DS 636 - DATA ANALYTICS WITH R PROGRAM,004,12241,T,2:30 PM - 5:20 PM,TIER LECT 2,Open,43,15,"Bhavsar, Jaini",Face-to-Face,3,Book,Students must bring their own device for this ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,DS 792B - PRE-DOCTORAL RESEARCH,606,12378,,TBA,,Open,19,0,"Wu, Chase",Face-to-Face,3,Book,See department for permission to enroll.
127,DS 792B - PRE-DOCTORAL RESEARCH,610,12379,,TBA,,Open,19,0,"Dasgupta, Aritra",Face-to-Face,3,Book,See department for permission to enroll.
128,DS 792B - PRE-DOCTORAL RESEARCH,616,12380,,TBA,,Open,19,1,"Phan, Hai",Face-to-Face,3,Book,See department for permission to enroll.
129,DS 792B - PRE-DOCTORAL RESEARCH,624,12381,,TBA,,Open,19,0,"Wang, Guiling",Face-to-Face,3,Book,See department for permission to enroll.


In [37]:
df.describe()

,Course_name,Section,CRN,Days,Times,Location,Status,Max,Now,Instructor,Delivery Mode,Credits,Info,Comments
count,131,131,131,131,131,131,131,131,131,131,131,131,131,131
unique,17,27,131,8,8,12,1,7,20,25,2,3,1,6
top,DS 792B - PRE-DOCTORAL RESEARCH,610,16164,,TBA,,Open,19,0,"Roshan, Usman",Face-to-Face,3,Book,See department for permission to enroll.
freq,18,7,1,115,115,119,131,110,104,8,127,112,131,110


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131 entries, 0 to 130
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Course_name    131 non-null    object
 1   Section        131 non-null    object
 2   CRN            131 non-null    object
 3   Days           131 non-null    object
 4   Times          131 non-null    object
 5   Location       131 non-null    object
 6   Status         131 non-null    object
 7   Max            131 non-null    object
 8   Now            131 non-null    object
 9   Instructor     131 non-null    object
 10  Delivery Mode  131 non-null    object
 11  Credits        131 non-null    object
 12  Info           131 non-null    object
 13  Comments       131 non-null    object
dtypes: object(14)
memory usage: 14.5+ KB


In [42]:


# Ensure 'Max' and 'Now' are numeric
df['Max'] = df['Max'].astype(int)
df['Now'] = df['Now'].astype(int)

# Add Seats Available column
df['Seats Available'] = df['Max'] - df['Now']

# Group data by Instructor and compute totals
instructor_summary = df.groupby('Instructor').agg({
    'Max': 'sum',             # Total Maximum Seats
    'Now': 'sum',             # Total Seats Taken
    'Seats Available': 'sum'  # Total Seats Available
}).reset_index()

# Rename columns for clarity
instructor_summary.columns = ['Instructor', 'Max Seats', 'Seats Taken', 'Seats Available']

# Display the summary DataFrame
print("Instructor Summary:")
instructor_summary


In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131 entries, 0 to 130
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Course_name    131 non-null    object
 1   Section        131 non-null    object
 2   CRN            131 non-null    object
 3   Days           131 non-null    object
 4   Times          131 non-null    object
 5   Location       131 non-null    object
 6   Status         131 non-null    object
 7   Max            131 non-null    int64 
 8   Now            131 non-null    int64 
 9   Instructor     131 non-null    object
 10  Delivery Mode  131 non-null    object
 11  Credits        131 non-null    object
 12  Info           131 non-null    object
 13  Comments       131 non-null    object
dtypes: int64(2), object(12)
memory usage: 14.5+ KB


In [44]:
df = df[~df['Course_name'].str.contains('DS 637')]
df = df[~df['Course_name'].str.contains('DS 650')]
df = df[~df['Course_name'].str.contains('DS 669')]
df = df[~df['Course_name'].str.contains('DS 675')]
df = df[~df['Course_name'].str.contains('DS 677')]
df = df[~df['Course_name'].str.contains('DS 488')]
df = df[~df['Course_name'].str.contains('DS 680')]

df

,Course_name,Section,CRN,Days,Times,Location,Status,Max,Now,Instructor,Delivery Mode,Credits,Info,Comments
0,DS 340 - FUND. AND PRIN. OF DS,004,16164,MW,10:00 AM - 11:20 AM,CKB 120,Open,38,11,"Bhavsar, Jaini",Face-to-Face,3,Book,
3,DS 636 - DATA ANALYTICS WITH R PROGRAM,002,12240,M,1:00 PM - 3:50 PM,KUPF 210,Open,43,7,"Bhavsar, Jaini",Face-to-Face,3,Book,Students must bring their own device for this ...
4,DS 636 - DATA ANALYTICS WITH R PROGRAM,004,12241,T,2:30 PM - 5:20 PM,TIER LECT 2,Open,43,15,"Bhavsar, Jaini",Face-to-Face,3,Book,Students must bring their own device for this ...
5,DS 636 - DATA ANALYTICS WITH R PROGRAM,006,12242,W,11:30 AM - 2:20 PM,KUPF 203,Open,43,26,"Bhavsar, Jaini",Face-to-Face,3,Book,Students must bring their own device for this ...
6,DS 636 - DATA ANALYTICS WITH R PROGRAM,008,12243,R,2:30 PM - 5:20 PM,KUPF 210,Open,43,3,"Mani, Kumar",Face-to-Face,3,Book,Students must bring their own device for this ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,DS 792B - PRE-DOCTORAL RESEARCH,606,12378,,TBA,,Open,19,0,"Wu, Chase",Face-to-Face,3,Book,See department for permission to enroll.
127,DS 792B - PRE-DOCTORAL RESEARCH,610,12379,,TBA,,Open,19,0,"Dasgupta, Aritra",Face-to-Face,3,Book,See department for permission to enroll.
128,DS 792B - PRE-DOCTORAL RESEARCH,616,12380,,TBA,,Open,19,1,"Phan, Hai",Face-to-Face,3,Book,See department for permission to enroll.
129,DS 792B - PRE-DOCTORAL RESEARCH,624,12381,,TBA,,Open,19,0,"Wang, Guiling",Face-to-Face,3,Book,See department for permission to enroll.


In [45]:
df[['Instructor','Max', 'Now']].groupby(['Instructor']).sum().tail(50)


,Max,Now
Instructor,,
"Afkhami, Shahriar",114,0
"Bader, David",144,4
"Basu Roy, Senjuti",114,0
"Bhavsar, Jaini",167,59
"Dasgupta, Aritra",114,1
"Du, Mengnan",114,2
"Geller, James",114,2
"Jin, Huiran",114,0
"Kaur, Ravneet",80,63
